In [2]:
import h2o
import pandas as pd
import numpy as np

In [3]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_171"; Java(TM) SE Runtime Environment (build 1.8.0_171-b11); Java HotSpot(TM) 64-Bit Server VM (build 25.171-b11, mixed mode)
  Starting server from /Users/raj/anaconda3/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/gl/wb7rqfgd3v708q5m0ydj3pkc0000gq/T/tmpifvav8cw
  JVM stdout: /var/folders/gl/wb7rqfgd3v708q5m0ydj3pkc0000gq/T/tmpifvav8cw/h2o_raj_started_from_python.out
  JVM stderr: /var/folders/gl/wb7rqfgd3v708q5m0ydj3pkc0000gq/T/tmpifvav8cw/h2o_raj_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,03 secs
H2O cluster timezone:,Asia/Kolkata
H2O data parsing timezone:,UTC
H2O cluster version:,3.20.0.2
H2O cluster version age:,9 days
H2O cluster name:,H2O_from_python_raj_wlfjtp
H2O cluster total nodes:,1
H2O cluster free memory:,3.556 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


In [37]:
url =  "http://h2o-public-test-data.s3.amazonaws.com/smalldata/airlines/allyears2k_headers.zip"
data=h2o.import_file(url)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [48]:
data.head()

Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,IsArrDelayed,IsDepDelayed
1987,10,14,3,741,730,912,849,PS,1451,NA,91,79,nan,23,11,SAN,SFO,447,nan,nan,0,NA,0,nan,nan,nan,nan,nan,YES,YES
1987,10,15,4,729,730,903,849,PS,1451,NA,94,79,nan,14,-1,SAN,SFO,447,nan,nan,0,NA,0,nan,nan,nan,nan,nan,YES,NO
1987,10,17,6,741,730,918,849,PS,1451,NA,97,79,nan,29,11,SAN,SFO,447,nan,nan,0,NA,0,nan,nan,nan,nan,nan,YES,YES
1987,10,18,7,729,730,847,849,PS,1451,NA,78,79,nan,-2,-1,SAN,SFO,447,nan,nan,0,NA,0,nan,nan,nan,nan,nan,NO,NO
1987,10,19,1,749,730,922,849,PS,1451,NA,93,79,nan,33,19,SAN,SFO,447,nan,nan,0,NA,0,nan,nan,nan,nan,nan,YES,YES
1987,10,21,3,728,730,848,849,PS,1451,NA,80,79,nan,-1,-2,SAN,SFO,447,nan,nan,0,NA,0,nan,nan,nan,nan,nan,NO,NO
1987,10,22,4,728,730,852,849,PS,1451,NA,84,79,nan,3,-2,SAN,SFO,447,nan,nan,0,NA,0,nan,nan,nan,nan,nan,YES,NO
1987,10,23,5,731,730,902,849,PS,1451,NA,91,79,nan,13,1,SAN,SFO,447,nan,nan,0,NA,0,nan,nan,nan,nan,nan,YES,YES
1987,10,24,6,744,730,908,849,PS,1451,NA,84,79,nan,19,14,SAN,SFO,447,nan,nan,0,NA,0,nan,nan,nan,nan,nan,YES,YES
1987,10,25,7,729,730,851,849,PS,1451,NA,82,79,nan,2,-1,SAN,SFO,447,nan,nan,0,NA,0,nan,nan,nan,nan,nan,YES,NO


In [ ]:
train,valid,test=data.split_frame([0.8,0.1],seed=69)


In [39]:
print("%d/%d/%d" %(train.nrows,valid.nrows,test.nrows))


35255/4272/4451


In [40]:
train2 =data[1:35255,:]
train2=h2o.assign(train2,'first35255')

In [41]:
h2o.ls()

,key
0,allyears2k_headers.hex
1,allyears2k_headers1.hex
2,allyears2k_headers2.hex
3,allyears2k_headers3.hex
4,first35255
5,py_10_sid_80de_splitter
6,py_17_sid_80de
7,py_18_sid_80de_splitter
8,py_1_sid_80de_splitter
9,py_25_sid_80de


In [42]:
data.ncol

31

In [43]:
dates=data[:,0:4]
dates

Year,Month,DayofMonth,DayOfWeek
1987,10,14,3
1987,10,15,4
1987,10,17,6
1987,10,18,7
1987,10,19,1
1987,10,21,3
1987,10,22,4
1987,10,23,5
1987,10,24,6
1987,10,25,7


In [44]:
airports= data[:,['Origin','Dest']]

In [46]:
dates.ncol

4

In [47]:
airports.ncol

2

##### use cbind to join ("bind") columns

In [53]:
a_and_d=airports.cbind(dates)

In [54]:
a_and_d

Origin,Dest,Year,Month,DayofMonth,DayOfWeek
SAN,SFO,1987,10,14,3
SAN,SFO,1987,10,15,4
SAN,SFO,1987,10,17,6
SAN,SFO,1987,10,18,7
SAN,SFO,1987,10,19,1
SAN,SFO,1987,10,21,3
SAN,SFO,1987,10,22,4
SAN,SFO,1987,10,23,5
SAN,SFO,1987,10,24,6
SAN,SFO,1987,10,25,7


Use rbind to join rows

In [55]:
restored_data=train.rbind([valid,test])

In [56]:
restored_data

Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,IsArrDelayed,IsDepDelayed
1987,10,14,3,741,730,912,849,PS,1451,NA,91,79,nan,23,11,SAN,SFO,447,nan,nan,0,NA,0,nan,nan,nan,nan,nan,YES,YES
1987,10,15,4,729,730,903,849,PS,1451,NA,94,79,nan,14,-1,SAN,SFO,447,nan,nan,0,NA,0,nan,nan,nan,nan,nan,YES,NO
1987,10,17,6,741,730,918,849,PS,1451,NA,97,79,nan,29,11,SAN,SFO,447,nan,nan,0,NA,0,nan,nan,nan,nan,nan,YES,YES
1987,10,18,7,729,730,847,849,PS,1451,NA,78,79,nan,-2,-1,SAN,SFO,447,nan,nan,0,NA,0,nan,nan,nan,nan,nan,NO,NO
1987,10,22,4,728,730,852,849,PS,1451,NA,84,79,nan,3,-2,SAN,SFO,447,nan,nan,0,NA,0,nan,nan,nan,nan,nan,YES,NO
1987,10,23,5,731,730,902,849,PS,1451,NA,91,79,nan,13,1,SAN,SFO,447,nan,nan,0,NA,0,nan,nan,nan,nan,nan,YES,YES
1987,10,24,6,744,730,908,849,PS,1451,NA,84,79,nan,19,14,SAN,SFO,447,nan,nan,0,NA,0,nan,nan,nan,nan,nan,YES,YES
1987,10,25,7,729,730,851,849,PS,1451,NA,82,79,nan,2,-1,SAN,SFO,447,nan,nan,0,NA,0,nan,nan,nan,nan,nan,YES,NO
1987,10,26,1,735,730,904,849,PS,1451,NA,89,79,nan,15,5,SAN,SFO,447,nan,nan,0,NA,0,nan,nan,nan,nan,nan,YES,YES
1987,10,28,3,741,725,919,855,PS,1451,NA,98,90,nan,24,16,SAN,SFO,447,nan,nan,0,NA,0,nan,nan,nan,nan,nan,YES,YES


In [57]:
data.dim

[43978, 31]

In [59]:
restored_data[:,0:4].head()

Year,Month,DayofMonth,DayOfWeek
1987,10,14,3
1987,10,15,4
1987,10,17,6
1987,10,18,7
1987,10,22,4
1987,10,23,5
1987,10,24,6
1987,10,25,7
1987,10,26,1
1987,10,28,3


In [60]:
data[:,0:4].head()

Year,Month,DayofMonth,DayOfWeek
1987,10,14,3
1987,10,15,4
1987,10,17,6
1987,10,18,7
1987,10,19,1
1987,10,21,3
1987,10,22,4
1987,10,23,5
1987,10,24,6
1987,10,25,7


In [61]:
train[:,0:4].head()

Year,Month,DayofMonth,DayOfWeek
1987,10,14,3
1987,10,15,4
1987,10,17,6
1987,10,18,7
1987,10,22,4
1987,10,23,5
1987,10,24,6
1987,10,25,7
1987,10,26,1
1987,10,28,3


Use h2o.merge() to join two tables when they have at least one column in common

Unlike cbind() they can have different number of rows, Unlike rbind() they can have different number of columns

In [80]:
A= h2o.H2OFrame(
pd.DataFrame({
    'a':range(1,11),
    'b':range(11,21),
    'c':range(21,31)
}),
destination_frame="A"
)

/Users/raj/anaconda3/lib/python3.6/site-packages/h2o/utils/shared_utils.py:177: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  data = _handle_python_lists(python_obj.as_matrix().tolist(), -1)[1]


Parse progress: |█████████████████████████████████████████████████████████| 100%


In [81]:
A.as_data_frame()

,a,b,c
0,1,11,21
1,2,12,22
2,3,13,23
3,4,14,24
4,5,15,25
5,6,16,26
6,7,17,27
7,8,18,28
8,9,19,29
9,10,20,30


In [82]:
np.random.seed(123)
B= h2o.H2OFrame(
pd.DataFrame({
    'a':np.random.randint(6,13,20),
    'd':np.random.randint(6,13,20)
}),
destination_frame="B"
)

/Users/raj/anaconda3/lib/python3.6/site-packages/h2o/utils/shared_utils.py:177: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  data = _handle_python_lists(python_obj.as_matrix().tolist(), -1)[1]


Parse progress: |█████████████████████████████████████████████████████████| 100%


In [83]:
B.as_data_frame()

,a,d
0,12,6
1,11,7
2,12,9
3,8,12
4,10,11
5,8,10
6,12,6
7,7,6
8,9,10
9,8,7


In [69]:
M = A.merge(B)

In [71]:
M.as_data_frame()

,a,b,c,d
0,6,16,26,10
1,6,16,26,7
2,7,17,27,6
3,7,17,27,8
4,7,17,27,8
5,7,17,27,6
6,7,17,27,6
7,8,18,28,12
8,8,18,28,10
9,8,18,28,7


In [73]:
M= B.merge(A)

In [74]:
M.as_data_frame()

,a,d,b,c
0,6,10,16,26
1,6,7,16,26
2,7,6,17,27
3,7,8,17,27
4,7,8,17,27
5,7,6,17,27
6,7,6,17,27
7,8,12,18,28
8,8,10,18,28
9,8,7,18,28


In [ ]:
#Note: by_x and by_y don't work in Python (as of 3.14)
# AS as workaround we have modified these examples to have the other of by_x and by_y
# be an explicit list of common columns (i.e ["a"])

In [86]:
M = B.merge(A,by_x="a",by_y=["b"])

In [87]:
M.as_data_frame()

,a,d,a.1,c
0,11,7,1,21
1,12,6,2,22
2,12,9,2,22
3,12,6,2,22
4,12,10,2,22
5,12,11,2,22
6,12,9,2,22


In [88]:
M = B.merge(A,by_x="b",by_y=["a"])

H2OValueError: Column: b not in frame.

In [95]:
M = A.merge(B,by_x="a",by_y=["a"])

In [91]:
M.as_data_frame()

,a,b,c,d
0,6,16,26,10
1,6,16,26,7
2,7,17,27,6
3,7,17,27,8
4,7,17,27,8
5,7,17,27,6
6,7,17,27,6
7,8,18,28,12
8,8,18,28,10
9,8,18,28,7


In [92]:
M = A.merge(B,all_y=True)

In [93]:
M.as_data_frame()

,a,b,c,d
0,6,16.0,26.0,10
1,6,16.0,26.0,7
2,7,17.0,27.0,6
3,7,17.0,27.0,8
4,7,17.0,27.0,8
5,7,17.0,27.0,6
6,7,17.0,27.0,6
7,8,18.0,28.0,12
8,8,18.0,28.0,10
9,8,18.0,28.0,7
